In [1]:
!pip install face_recognition efficientnet tensorflow opencv-python-headless gradio --quiet


In [2]:
import os
import cv2
import numpy as np
import face_recognition
import gradio as gr
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import EfficientNetB4
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
import tensorflow as tf


In [3]:
import os
import cv2
import numpy as np
import random
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def load_images(folder, label, size=(244, 244)):
    data = []
    for filename in os.listdir(folder):
        path = os.path.join(folder, filename)
        img = cv2.imread(path)
        if img is not None:
            img = cv2.resize(img, size)
            data.append((img, label))
    return data

# Load data
real_path = r"D:\online_set\real_and_fake_face\real"
fake_path = r"D:\online_set\real_and_fake_face\fake"

real_data = load_images(real_path, 0)
fake_data = load_images(fake_path, 1)
data = real_data + fake_data

# Shuffle
random.shuffle(data)

# Convert to NumPy with reduced memory usage
X = np.array([item[0] for item in data], dtype=np.float32) / 255.0
y = np.array([item[1] for item in data])
y = to_categorical(y, num_classes=2)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Augmentation
datagen = ImageDataGenerator(
    rotation_range=10,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)
datagen.fit(X_train)


In [4]:
base_model = EfficientNetB4(weights='imagenet', include_top=False, input_shape=(244, 244, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(2, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(
    datagen.flow(X_train, y_train, batch_size=16),
    epochs=5,
    validation_data=(X_test, y_test)
)
model.save("deepfake_detector_efficientnetb4.h5")

D:\ana\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
106/106 ━━━━━━━━━━━━━━━━━━━━ 132s 1s/step - accuracy: 0.5005 - loss: 0.7172 - val_accuracy: 0.4775 - val_loss: 0.7097
Epoch 2/5
106/106 ━━━━━━━━━━━━━━━━━━━━ 112s 1s/step - accuracy: 0.4972 - loss: 0.7054 - val_accuracy: 0.4775 - val_loss: 0.7004
Epoch 3/5
106/106 ━━━━━━━━━━━━━━━━━━━━ 115s 1s/step - accuracy: 0.5031 - loss: 0.7102 - val_accuracy: 0.5225 - val_loss: 0.6940
Epoch 4/5
106/106 ━━━━━━━━━━━━━━━━━━━━ 112s 1s/step - accuracy: 0.4753 - loss: 0.7197 - val_accuracy: 0.5201 - val_loss: 0.6918
Epoch 5/5
106/106 ━━━━━━━━━━━━━━━━━━━━ 115s 1s/step - accuracy: 0.4788 - loss: 0.7126 - val_accuracy: 0.4775 - val_loss: 0.6958


In [5]:
for layer in base_model.layers[-20:]:
    layer.trainable = True

# Step 4: Recompile with a lower learning rate for fine-tuning
model.compile(optimizer=tf.keras.optimizers.Adam(1e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Step 5: Continue training (fine-tuning)
model.fit(
    datagen.flow(X_train, y_train, batch_size=16),
    epochs=5,
    validation_data=(X_test, y_test)
)

# Step 6: Save the fine-tuned model
model.save("deepfake_detector_efficientnetb4_finetuned.h5")

Epoch 1/5
106/106 ━━━━━━━━━━━━━━━━━━━━ 146s 1s/step - accuracy: 0.5009 - loss: 0.6999 - val_accuracy: 0.5177 - val_loss: 0.6916
Epoch 2/5
106/106 ━━━━━━━━━━━━━━━━━━━━ 129s 1s/step - accuracy: 0.5275 - loss: 0.6899 - val_accuracy: 0.5225 - val_loss: 0.6947
Epoch 3/5
106/106 ━━━━━━━━━━━━━━━━━━━━ 125s 1s/step - accuracy: 0.5043 - loss: 0.6950 - val_accuracy: 0.5083 - val_loss: 0.6911
Epoch 4/5
106/106 ━━━━━━━━━━━━━━━━━━━━ 126s 1s/step - accuracy: 0.5191 - loss: 0.6938 - val_accuracy: 0.5059 - val_loss: 0.6916
Epoch 5/5
106/106 ━━━━━━━━━━━━━━━━━━━━ 129s 1s/step - accuracy: 0.5051 - loss: 0.6947 - val_accuracy: 0.5296 - val_loss: 0.6911


In [6]:
from pathlib import Path
from PIL import Image
import face_recognition

known_faces = []
known_names = []

dataset_path = Path(r"D:\new_project\face_dataset")

# Accept common image file extensions
valid_extensions = {'.jpg', '.jpeg', '.png', '.bmp'}

for img_path in dataset_path.rglob("*"):
    if img_path.suffix.lower() not in valid_extensions:
        continue

    try:
        image = face_recognition.load_image_file(str(img_path))
        encodings = face_recognition.face_encodings(image)

        if encodings:
            known_faces.append(encodings[0])
            name = img_path.parent.name  # Use folder name as label
            known_names.append(name)
        else:
            print(f"No face found in: {img_path.name}")

    except Exception as e:
        print(f"Error processing {img_path.name}: {e}")


No face found in: elon musk100_1525.jpg
No face found in: elon musk114_1535.jpg
No face found in: elon musk145_1549.jpg
No face found in: elon musk147_1551.jpg
No face found in: elon musk209_1587.jpg
No face found in: elon musk53_1631.jpg
No face found in: jeff bezos163_2070.jpg
No face found in: Taylor Swift119_4562.jpg
No face found in: Taylor Swift144_4572.jpg
No face found in: Taylor Swift189_4606.jpg
No face found in: Taylor Swift195_4607.jpg
No face found in: Taylor Swift212_4617.jpg
No face found in: Tom Cruise124_1046.jpg
No face found in: Tom Cruise133_1053.jpg
No face found in: Tom Cruise146_1061.jpg
No face found in: Tom Cruise15_1065.jpg
No face found in: Tom Cruise217_1123.jpg
No face found in: Tom Cruise246_1144.jpg
No face found in: Tom Cruise63_1177.jpg
No face found in: Tom Cruise80_1193.jpg
No face found in: Tom Cruise80_1194.jpg
No face found in: Tom Cruise86_1201.jpg
No face found in: tom ellis106_4255.jpg
No face found in: tom ellis133_4279.jpg
No face found in: to

In [19]:
from tensorflow.keras.models import load_model

# Load your trained model before prediction
model = load_model("deepfake_detector_efficientnetb4_finetuned.h5")

def predict_and_recognize(image, is_webcam=False):
    try:
        # Deepfake Detection (skip if webcam input)
        if not is_webcam:
            image_resized = cv2.resize(image, (244, 244)) / 255.0
            input_tensor = np.expand_dims(image_resized, axis=0)

            pred = model.predict(input_tensor)[0]
            predicted_class = np.argmax(pred)
            confidence = round(float(np.max(pred)) * 100, 2)

            is_real = predicted_class == 0 and confidence >= 51.0
            result = "Real" if is_real else "Fake"
            full_result = f"{result} Face ({confidence}% Confidence)"
        else:
            # Force real for webcam
            full_result = "Real Face (Captured via Webcam)"

        # Face recognition (always performed)
        rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        face_locations = face_recognition.face_locations(rgb_image)
        if not face_locations:
            return f"{full_result} - No face detected for recognition"

        face_encodings = face_recognition.face_encodings(rgb_image, face_locations)
        if not face_encodings:
            return f"{full_result} - No encodings could be computed"

        for face_encoding in face_encodings:
            matches = face_recognition.compare_faces(known_faces, face_encoding)
            face_distances = face_recognition.face_distance(known_faces, face_encoding)
            best_match_index = np.argmin(face_distances)

            if matches[best_match_index]:
                name = known_names[best_match_index]
                return f"{full_result} - Identified as: {name}"

        return f"{full_result} - Face Not Recognized"

    except Exception as e:
        return f"Error during prediction: {str(e)}"




In [20]:
interface = gr.Interface(fn=predict_and_recognize,
                         inputs=gr.Image(type="numpy"),
                         outputs="text",
                         title="Deepfake Detection + Face Recognition",
                         description="Upload a face image. It will classify as real/fake and identify the person if real.")

interface.launch(share=True)


* Running on local URL:  http://127.0.0.1:7866

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.
